## Парсер для списков поступающих на программы магистратуры
Код формирует xlsx таблицу со списком поступающих.
Вам необходимо лишь заменить ссылку в requests.get() на нужную.

Для этого:

* Переходите на страницу https://abit.itmo.ru/ratings/master

* Выбираете интересующую программу

* Копируете ссылку и вставляете в код, запускаете

* В файлах появляется эксель таблица, которую можно скачать и отфильтровать

In [ ]:
import re
import requests
import pandas as pd

# замените ссылку в ковычках на страницу списков подавших нужной программы
req = requests.get("https://abit.itmo.ru/rating/master/budget/1943")
# считываем текст HTML-документа
src = req.text

def process_string(src):
    # Удаляем всё перед первым "exam_type"
    first_exam_type_index = src.find("exam_type")
    if first_exam_type_index == -1:
        return []  # "exam_type" не найден

    cleaned_data = src[first_exam_type_index:]

    # Разделяем по строкам, начинающимся с "exam_type"
    result = re.split(r'(?=exam_type)', cleaned_data)

    return result
result = process_string(src)

# Создаём DataFrame из результата
df = pd.DataFrame(result, columns=['exam_data'])


# Функция для обработки каждой строки
def process_raw_data(raw_data):
    # Регулярные выражения для извлечения значений
    pattern = re.compile(r'exam_type":"(.*?)".*?diploma_average":(.*?),.*?position":(.*?),.*?priority":(.*?),.*?total_scores":(.*?),.*?is_send_original":(.*?),.*?snils":"(.*?)"')
    match = pattern.search(raw_data)
    if match:
        return {
            'exam_type': match.group(1),
            'diploma_average': float(match.group(2)),
            'position': int(match.group(3)),
            'priority': int(match.group(4)),
            'total_scores': float(match.group(5)),
            'is_send_original': match.group(6) == 'true',
            'snils': match.group(7)
        }
    else:
        return None

# Применение функции ко всем строкам
processed_data = df['exam_data'].apply(process_raw_data).dropna()

# Создание нового DataFrame
new_df = pd.DataFrame(processed_data.tolist())

# Установка 'position' в качестве индекса
new_df.set_index('position', inplace=True)
# Вывод нового DataFrame
new_df.to_excel('Списки.xlsx')